In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from tensorflow.keras.layers import Activation
from tensorflow.keras.activations import gelu
import warnings
warnings.filterwarnings('ignore')

# === SEQUENCE CREATION FUNCTION ===
def create_sequences(data, features, target, sequence_length):
    """
    Convert time series data into sequences for LSTM training
    Returns:
        X: 3D array of sequences (samples, time steps, features)
        y: 1D array of target values
        dates: Corresponding dates for each sequence
    """
    X, y, dates = [], [], []
    data_values = data[features].values
    target_values = data[target].values
    date_values = data.index.values
    
    for i in range(sequence_length, len(data)):
        X.append(data_values[i-sequence_length:i])
        y.append(target_values[i])
        dates.append(date_values[i])
    
    return np.array(X), np.array(y), np.array(dates)

# === SETTINGS ===
TARGET = "future_return_1d"
SEQUENCE_LENGTH = 30
EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 0.003
PURGE_DAYS = 5
DROPOUT_RATE = 0.2
L2_REG = 0.001

top_features = [
    'open',
    'high',
    'low',
    'close',
    'volume',
    'rsi',
    'ema_short',
    'ema_long',
    'volatility_atr',
    'bb_width',
    'obv',
    'volume_norm',
    'macd',
    'macd_signal',
    'macd_hist',
    'return_1d',
    'return_3d',
    'return_7d',
    'adx',
    'hma_14',
    'vwap',
    'cmf',
    'sentiment_news_z',
    'sentiment_twitter_z'
]

# === DATA LOADING ===
df = pd.read_csv("/kaggle/input/final-features/BTCUSDT_features_final_sentiment.csv", parse_dates=["timestamp"])
df.set_index("timestamp", inplace=True)

# === DROP NaNs ===
df.dropna(subset=top_features + [TARGET], inplace=True)

# === BUILD LSTM MODEL ===
def build_lstm(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(
            128, return_sequences=True, input_shape=input_shape,
            dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE / 2,
            kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
        ),
        tf.keras.layers.LSTM(
            64, return_sequences=False,
            dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE / 2,
            kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
        ),
        tf.keras.layers.Dense(
            32, activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
        ),
        tf.keras.layers.Dropout(DROPOUT_RATE),
        tf.keras.layers.Dense(
            16, activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
        ),
        tf.keras.layers.Dropout(DROPOUT_RATE),
        tf.keras.layers.Dense(1, activation='linear')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='mse',
        metrics=['mae']
    )
    return model



# === FEATURE IMPORTANCE ===
def calculate_rf_importance(X, y, features):
    X_flat = X.reshape(X.shape[0], -1)
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_flat, y)
    importances = rf.feature_importances_
    grouped = {}
    for i, feat in enumerate(features):
        grouped[feat] = np.sum(importances[i::len(features)])
    return grouped

# === METRICS ===
def evaluate_metrics(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    corr, _ = pearsonr(y_true, y_pred)
    hit_rate = np.mean(np.sign(y_true) == np.sign(y_pred))
    return {"r2": r2, "mae": mae, "rmse": rmse, "corr": corr, "hit_rate": hit_rate}

# === WINDOWS ===
WINDOWS = [
    {"name": "W1", "train_end": "2021-09-30", "test_start": "2021-10-10", "test_end": "2021-12-31"},
    {"name": "W2", "train_end": "2021-11-30", "test_start": "2021-12-10", "test_end": "2022-02-28"},
]

results = []
all_feature_importance = {}

# === MAIN TRAINING LOOP ===
for w in WINDOWS:
    print(f"\n🚀 {w['name']} | Training until {w['train_end']} | Testing from {w['test_start']} to {w['test_end']}")

    train_end = pd.to_datetime(w["train_end"])
    test_start = pd.to_datetime(w["test_start"])
    test_end = pd.to_datetime(w["test_end"])
    purge_start = test_start - timedelta(days=PURGE_DAYS)

    train = df[df.index <= purge_start]
    test = df[(df.index >= test_start) & (df.index <= test_end)]

    # Scaling
    scaler = StandardScaler()
    train_scaled = train.copy()
    test_scaled = test.copy()
    train_scaled[top_features] = scaler.fit_transform(train[top_features])
    test_scaled[top_features] = scaler.transform(test[top_features])

    # Sequences
    X_train, y_train, _ = create_sequences(train_scaled, top_features, TARGET, SEQUENCE_LENGTH)
    X_test, y_test, test_dates = create_sequences(test_scaled, top_features, TARGET, SEQUENCE_LENGTH)

    if len(X_train) < 50 or len(X_test) < 10:
        print(" Not enough data, skipping...")
        continue

    # Scale target
    target_scaler = StandardScaler()
    y_train_scaled = target_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = target_scaler.transform(y_test.reshape(-1, 1)).flatten()

    tf.keras.backend.clear_session()
    model = build_lstm((SEQUENCE_LENGTH, len(top_features)))

    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint(f"model_{w['name']}.h5", save_best_only=True)
    ]

    history = model.fit(
        X_train, y_train_scaled,
        validation_split=0.1,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1,
        callbacks=callbacks,
        shuffle=True
    )

    preds_scaled = model.predict(X_test).flatten()
    preds = target_scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()

    metrics = evaluate_metrics(y_test, preds)
    metrics["ic"], _ = spearmanr(y_test, preds)
    hit_rate_scaled = np.mean(np.sign(y_test_scaled) == np.sign(preds_scaled))
    print(f"🔁 Hitrate (scaled): {hit_rate_scaled:.4f} | (inverse): {metrics['hit_rate']:.4f}")
    results.append({"window": w["name"], **metrics})

    importance = calculate_rf_importance(X_train, y_train_scaled, top_features)
    all_feature_importance[w["name"]] = importance

    # Plot learning curve
    plt.figure(figsize=(8, 4))
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Val Loss")
    plt.title(f"Learning Curve - {w['name']}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Plot residuals
    residuals = y_test - preds
    plt.figure(figsize=(8, 4))
    plt.hist(residuals, bins=20, edgecolor='black', color='skyblue', alpha=0.7)
    plt.axvline(0, color='red', linestyle='--')
    plt.title(f"Residuals Distribution - {w['name']}")
    plt.xlabel("Residual")
    plt.ylabel("Frequency")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Print top features
    print(f"Top 3 features in {w['name']}:")
    for feat, imp in sorted(importance.items(), key=lambda x: x[1], reverse=True)[:3]:
        print(f"  {feat}: {imp:.4f}")

# === COMBINED RESULTS ===
# Combined scatter plot
all_preds, all_true, all_labels = [], [], []
for i, w in enumerate(WINDOWS):
    w_name = w["name"]
    test = df[w["test_start"]:w["test_end"]]
    test_scaled = test.copy()
    test_scaled[top_features] = scaler.transform(test[top_features])
    X_test, y_test, _ = create_sequences(test_scaled, top_features, TARGET, SEQUENCE_LENGTH)
    model = tf.keras.models.load_model(f"model_{w_name}.h5", compile=False)
    preds = model.predict(X_test).flatten()
    preds = target_scaler.inverse_transform(preds.reshape(-1, 1)).flatten()
    all_preds.extend(preds)
    all_true.extend(y_test)
    all_labels.extend([w_name] * len(y_test))

plt.figure(figsize=(9, 6))
sns.scatterplot(x=all_true, y=all_preds, hue=all_labels, alpha=0.6, palette="Set2")
min_val, max_val = min(min(all_true), min(all_preds)), max(max(all_true), max(all_preds))
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=1.5, label="Perfect Prediction")
plt.xlabel("Actual 1d Return")
plt.ylabel("Predicted 1d Return")
plt.title("Actual vs Predicted Returns (All Windows)")
plt.grid(True, alpha=0.3)
plt.legend(title="Window")
plt.tight_layout()
plt.show()

# Results dataframe
results_df = pd.DataFrame(results)
print("\n📊 Summary:")
print(results_df.round(4))

# Feature importance heatmap
importance_df = pd.DataFrame(all_feature_importance).fillna(0)
mean_importance = importance_df.mean(axis=1)
sorted_features = mean_importance.sort_values(ascending=False).index
importance_df_sorted = importance_df.loc[sorted_features]

plt.figure(figsize=(10, 6))
sns.heatmap(importance_df_sorted, annot=True, fmt=".3f", cmap="Blues")
plt.title("Feature Importance per Window (Ranked)")
plt.tight_layout()
plt.show()

# Average feature importance
plt.figure(figsize=(8, 4))
mean_importance.sort_values().plot(kind='barh', color='steelblue')
plt.title("Average Feature Importance (All Windows)")
plt.xlabel("Mean Importance")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()